In [53]:
import os
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
from scipy.io import wavfile
from python_speech_features import mfcc
import librosa
from librosa.feature import melspectrogram
import pickle

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from tensorflow.keras.utils import to_categorical


from buildfeats import build_rand_feat, check_data
from model import get_conv_model
from cfg import Config


Building features for train


NameError: name 'n_samples' is not defined

In [37]:
config = Config()

In [13]:
##experimenting just on the 'curated' train set
df = pd.read_csv('data/train/roadsound_labels.csv', index_col=0)
df.set_index('fname', inplace=True)
for f in df.index:
    rate, signal = wavfile.read('clean/'+f)
    df.at[f, 'length'] = signal.shape[0]/rate
df.shape

(297, 2)

In [14]:
df = df[df.length > config.step/rate]
df.shape

(297, 2)

In [38]:
classes = list(np.unique(df.labels))
class_dist = df.groupby(['labels'])['length'].mean()
n_samples = 2 * int(df['length'].sum() / 0.1) # 40 * total length of audio
prob_dist = class_dist / class_dist.sum()
choices = np.random.choice(class_dist.index, p=prob_dist)

In [39]:
df, test_df, _ , _ = train_test_split(df, df.labels)

In [41]:
X = []
y = []
split='train'
n_samples = 1000
_min, _max = float('inf'), -float('inf')

print('Building features for '+split)
for _ in tqdm(range(n_samples)):
    rand_class = np.random.choice(classes, p=prob_dist)
    file = np.random.choice(df[df.labels == rand_class].index)
    rate, wav = wavfile.read('clean/'+file)
    rand_index = np.random.randint(0, wav.shape[0] - config.step)
    sample = wav[rand_index:rand_index + config.step]
    X_sample = sample
    _min = min(np.amin(X_sample), _min)
    _max = max(np.amax(X_sample), _max)
    X.append(X_sample)
    y.append(classes.index(rand_class))
config.min = _min
config.max = _max
X, y = np.array(X), np.array(y)
X = (X - _min) / (_max - _min)
# X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1) ## not needed for raw signal
y = to_categorical(y)

Building features for train


In [42]:
print('X shape: ',X.shape)
print('y shape: ', y.shape)

X shape:  (1000, 3200)
y shape:  (1000, 5)


In [44]:
pca = PCA(n_components=20, random_state=1738)

In [47]:
X_pca = pca.fit_transform(X)

In [48]:
X_pca.shape

(1000, 20)

In [49]:
X_pca

array([[ 8.7662134e-03, -1.9523339e-02,  3.2622710e-02, ...,
         3.5260312e-02, -6.8924385e-03, -2.4721950e-02],
       [ 7.1668744e-02,  5.1486781e-03,  6.2997319e-02, ...,
        -2.9929197e-01,  3.6876384e-02,  1.9901377e-01],
       [-2.5725091e-02, -2.4770791e-02,  3.6925633e-02, ...,
        -2.0479662e-02, -5.8122035e-02, -2.5962362e-02],
       ...,
       [-1.3182645e-01,  3.4159555e-03, -7.8217924e-02, ...,
        -4.9240229e-01,  2.6852839e-02, -5.4261488e-01],
       [ 9.1471717e-02, -3.1634741e-02, -1.2360845e-02, ...,
        -7.0220634e-02, -2.8868730e-04,  7.9446882e-02],
       [-1.1637253e-02, -1.5255109e-02,  4.7989786e-03, ...,
        -3.9819963e-02, -1.7066717e-02, -1.0150173e-02]], dtype=float32)

In [51]:
input_shape = (None,20)

In [ ]:
model = get